In [23]:
import sys
sys.path.append("..")
from common_utils import set_data_home
set_data_home("/mnt/d")
from common_utils import DATA_HOME, join
from lstm.sales_data import Sales_Dataset
from torch.utils.data.dataloader import DataLoader
import torch

In [24]:
sd = Sales_Dataset(join(DATA_HOME, "sales_data"))
dl = DataLoader(sd, shuffle=True, batch_size=1)
len(sd)

# print(sd.O.describe())
# print(sd.TR.describe())
# print(sd.TS.describe())
# print(sd.S.describe())
# print(sd[4][0].shape)
# print(sd[4][1].shape)
# print(sd[4][0], sd[4][1])

KeyboardInterrupt: 

In [ ]:
from torch.nn import LSTM
from torch.nn import MSELoss
from torch import optim
import matplotlib.pyplot as plt

I, H = 71, 33
epochs = 10

model = LSTM(71, 33, batch_first=True).cuda()
loss = MSELoss()
adam = optim.Adam(model.parameters(), lr=0.001)
h, c = torch.randn(1,1, 33, dtype=torch.float32).cuda(), torch.randn(1,1, 33, dtype=torch.float32).cuda()
model.train()
losses = []
for _ in range(epochs):
    for X, y in dl:
        yhat, (_, _) = model(X, (h, c))
        l = loss(yhat, y.cuda())

        adam.zero_grad()
        l.backward()
        adam.step()
        losses.append(l.item())
plt.plot(range(len(dl)) * epochs, losses)
plt.show()

    




KeyboardInterrupt: 